<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls "/content/gdrive/My Drive/data"

Mall_Customers.csv  pdf_to_excel.pdf  pdf_to_excel.txt


In [3]:
pip install tika

  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
Successfully built tika


In [0]:
# Text parsing library
import tika
from tika import parser

# Deal with regular expression(text processing)
import re

# Handle the excel spreadsheet
import openpyxl

import os

tika.initVM()

# Read the ;PDF file
parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/sample_pdf_for_excel.pdf')
content = parsed["content"].strip()


In [0]:

f = open("/content/gdrive/My Drive/Colab Notebooks/data/pdf_content.txt", "w")
f.write(content)
f.close()

# content = f.open("/content/gdrive/My Drive/Colab Notebooks/data/pdf_content.txt", "f")
with open('/content/gdrive/My Drive/Colab Notebooks/data/pdf_content.txt', 'r') as myfile:
  content = myfile.read()

In [0]:
# Read the spreadsheet for later use
wb = openpyxl.load_workbook("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")
sheet = wb['Template']
# Starting writing the sheet beginning line#2
sheet_row_num = 2

# PDF columns
col_names = ["top_coat_date", "blast_primer_date"
             ,"dft1_ave", "dft1_5", "dft1_4", "dft1_3", "dft1_2", "dft1_1"
             ,"dft2_ave", "dft2_5", "dft2_4", "dft2_3", "dft2_2", "dft2_1"
             , "sys_tem"]

# Spreadsheet cells
cells = ["AL", "S", "AC", "AB", "AA", "Z", "Y", "X", "K", "J", "I", "H", "G", "F", "D"]

cols = {}

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  return val



In [15]:
s = re.sub(r"([0-9])_ ", r"\g<1> ", "502_1!18 UED-702 ]_ _112 _106_ 100 122 114 111_ 207 238 204 219 244 222 04-Nov-14 05-Nov-14")
re.sub(r" _([0-9])", r" \g<1>", s)

'502_1!18 UED-702 ]_ 112 106 100 122 114 111 207 238 204 219 244 222 04-Nov-14 05-Nov-14'

In [50]:
# c = content[:11000].strip()
c = content
# page_num_printed = 5479
page_num_printed = 5482
page_num = 4
skip = 11
total_pages = int(parsed["metadata"]["xmpTPg:NPages"].strip())+1
total_pages = 5

def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))

for page_num in range(4, total_pages):
  print("Page#%i"%page_num)
  print("Page num printed#%i"%page_num_printed)
  
  start_of_page = "Page " + str(page_num_printed) +" of 13699"
  end_of_page = "Page " + str(page_num_printed + 1) +" of 13699"
  page_content = c[c.find(start_of_page): c.find(end_of_page)]
  
#   print(page_content)
  
  try:
    p = re.compile("(A*VE\\s+DATE)", re.IGNORECASE)
    for m in p.finditer(page_content):
      start_of_table = m.start()+8

    end_of_table = re.search("(FINAL\\s+)*INSPECTION", page_content, flags = re.I)
    if end_of_table is None:
      end_of_table = re.search("(VISUAL\\s+)*INSPECTION", page_content, flags = re.I)
      skip = 29
    
    if end_of_table is None: 
      end_of_table = re.search("\\(*REMARK", page_content, flags = re.I)
      skip = 50
    
    end_of_table = end_of_table.start() - skip
    
    table = page_content[start_of_table:end_of_table]

    # Data cleansing
    table = table.replace("·","-")\
      .replace("•","")\
      .replace(" ' "," ")\
      .replace("l4", "14")\
      .replace(",", "")\
      .replace(" . ", " ")\
      .replace(" - ", " ")\
    

    table = re.sub("OS-([A-Z]{3})", r"05-\1", table,  flags=re.I)

    page_num_printed = page_num_printed + 1

#     print(table)
    
    # Parse table row by row
    for row in [x for x in table[2:].splitlines() if x]:
      row = row.strip()
      row = re.sub("-\\s*$", "", row)
  #     row = re.sub("^\\s*(\\w+|-)\\s+", "", row)
      row = re.sub("^.*?\\s+", "", row)
      row = re.sub("^\\s*\\w+\\s+", "", row)
  #     print(len(row))

      if len(row) > 1 and \
        hasNumbers(row):
        global r
        r = row
        i = 0
        print("-----")
        print(row)
        for col_name in col_names:
          col_val = get_col()
          cols[col_name] = col_val
          col_val = re.sub(r"([0-9]+)_", "\g<1>",col_val)
          col_val = re.sub(r"_([0-9]+)", "\g<1>",col_val)
          col_val = re.sub(r"\s+-([0-9]+)", " \g<1>",col_val)
          col_val = re.sub(r"([0-9]+)-\s+", "\g<1> ",col_val)
          
          
          print(col_val)
#         
# Write in spreadsheet
#           cell = cells[i] + str(sheet_row_num)    
#           sheet[cell].value= col_val
#       #     print("Value:{0}, cell:{1}".format(col_val, cell))
#           i = i + 1


#         r = re.sub("\\s+", "_", r)
#         cols["spool_no"] = r
#         cell ="D"+str(sheet_row_num)    
#         sheet[cell].value = r
#       #   print("Value:{0}, cell:{1}".format(r, cell))

#         cell ="C"+str(sheet_row_num)    
#         sheet[cell].value = page_num
#       #   print("Value:{0}, cell:{1}".format(page_num, cell))

#         sheet_row_num = sheet_row_num + 1

#       #   for k,v in cols.items():
#       #     print("{0}:{1}".format(k,v))

#       wb.save("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")
  except Exception:
    print("Exception on page%i" %page_num)
    traceback.print_exc()
    pass
    
    

Page#4
Page num printed#5482
-----
I C7EA-410 7 106 104 125 us 117 113 205 224 245 229 239 228 04-Nov-14 05-Nov-14
 05-Nov-14
 04-Nov-14
 228
 239
 229
 245
 224
 205
 113
 117
 us
 125
 104
 106
 7
-----
7EA-410 Inn- -'""-"" 7 114 12S 108 103 122 114 210 237 206 213 234 220 04-Nov-14 05-Nov-14 
 05-Nov-14
 04-Nov-14
 220
 234
 213
 206
 237
 210
 114
 122
 103
 108
 12S
 114
 7
-----
~2EM-SS2 7 102 108 120 124 112 113 204 229 2SO 221 248 230 04-Nov-14 05-Nov-14
 05-Nov-14
 04-Nov-14
 230
 248
 221
 2SO
 229
 204
 113
 112
 124
 120
 108
 102
 7
-----
IA2EL-SS2 AMOO<- <Ao.o 7 110 11S 124 102 116 113 209 233 202 22S 243 222 04-Nov-14 05-Nov-14
 05-Nov-14
 04-Nov-14
 222
 243
 22S
 202
 233
 209
 113
 116
 102
 124
 11S
 110
 7
-----
UEA-602 7 113 124 106 119 121 117 214 228 249 212 233 227 04-Nov-14 05-Nov-14
 05-Nov-14
 04-Nov-14
 227
 233
 212
 249
 228
 214
 117
 121
 119
 106
 124
 113
 7
-----
IA2EA-702. r<mR.1<Ro.m 7 101 112 119 123 111 113 203 236 207 228 2SO 225 04-Nov-14 05-Nov

In [0]:
pip install pypdfocr

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 2.6MB 8.0MB/s 
     |████████████████████████████████| 92kB 25.3MB/s 
     |████████████████████████████████| 81kB 23.8MB/s 
     |████████████████████████████████| 143kB 39.8MB/s 
ERROR: Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-0png968o/evernote/
